In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import operator
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

# Some info about what our questions are

## First we will clean the data

In [62]:
columns = ['Artist', 'Title', 'Album', '#_of_views', 'Release_date', '#_feat_artists', 
           'Artist_pop', 'Genre', 'Followers', 'Danceability', 'Track_pop', 'Explicit', 
           'Energy', 'Valence', 'Duration', 'Loudness', '#_words', 'Lyrics']
data = pd.read_csv('lyricDatasetUpdated.csv', names = columns)

Based on the greatest similarity between track titles, the data from Spotify was concatenated to the current Genius dataset. Now the dataset has additional features from Spotify. Some of the feature descriptions are grabbed straight from the Spotify API. The full list of features are as follow:<br>

1. **`Artist`** - Song artist<br>
DESCRIPTION: Name of artist
2. **`Title`** - Song title<br>
DESCRIPTION: Name of song
3. **`Album`** - Album Title<br>
DESCRIPTION: Name of album the song is from.
4. **`#_of_views`** - Number of pageviews for the lyric<br>
DESCRIPTION: Amount of views the lyric page got on the Genius website.
5. **`Release_date`** - Release date of song<br>
DESCRIPTION: The date the song was released
6. **`#_feat_artists`** - How many featured artists<br>
DESCRIPTION: The amount of artists featured on song
7. **`Artist_pop`** - Artist Popularity<br>
DESCRIPTION: How popular an artist is on Spotify. Range is 0-100.
8. **`Genre`** - Genre(s)<br>
DESCRIPTION: Genre the artist is considered as, separated by ' / '.
9. **`Followers`**<br>
DESCRIPTION: The amount of users following an artist on Spotify
10. **`Danceability`**<br>
DESCRIPTION: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. (Gotten from Spotify)
11. **`Track_pop`**<br>
DESCRIPTION: Track popularity is separate from artist popularity. It's the metric grabbed from Spotify on how popular a track is.
12. **`Explicit`**<br>
DESCRIPTION: If a song has lyrics on mature themes, or foul language, it is considered explicit. This is a binary value.
13. **`Energy`**<br>
DESCRIPTION: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
14. **`Valence`**<br>
DESCRIPTION: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
15. **`Duration`** - Duration (in Milliseconds)<br>
DESCRIPTION: The length of the song in milliseconds
16. **`Loudness`**<br>
DESCRIPTION: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
17. **`#_words`** - Total words in lyrics<br>
DESCRIPTION: How many words each lyric has.
18. **`Lyrics`**<br>
DESCRIPTION: The lyrics for the specific song.

In [63]:
data['Genre'].fillna('hip hop', inplace = True)

In [64]:
data.dropna(subset = ['Lyrics'], inplace = True)
data.reset_index(drop = True, inplace = True)

In [65]:
genres = data['Genre']
g_list = list(genres)
genre_list = ['country', 'hip hop', 'pop', 'rock', 'punk', 'metal', 
              'australian'] #Australian reggae is outlier

In [66]:
correct_guesses = {}
for index, genre in enumerate(g_list):
    if type(genre) == float:
        continue
    else:
        if len(genre) == 1:
            print(genre)
            split_genres = genre
        else:
            split_genres = genre.split(' / ') 
    for g in genre_list:
        correct = 0
        if len(split_genres) == 1 and split_genres[0] == 'rap':
            correct_guesses['hip hop'] = correct
        else:
            for split in split_genres:
                if g in split:
                    correct += 1
            correct_guesses[g] = correct
    
    correct_genre = max(correct_guesses, key=lambda key: correct_guesses[key])
    if correct_genre == 'australian':
        g_list[index] = 'pop'
    elif correct_genre == 'punk':
        g_list[index] = 'rock'
    else:
        g_list[index] = correct_genre
    correct_guesses.clear()

In [67]:
data['Genre'] = g_list

In [68]:
data['Lyrics'] = data['Lyrics'].astype(str)
data['Lyrics'] = data['Lyrics'].str.lower()
data['Lyrics'] = data['Lyrics'].str.translate(string.punctuation)
data['Lyrics'] = data['Lyrics'].str.replace('+', ' ').replace('\\', '').replace('"', '')

In [69]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = []
analyzer = SentimentIntensityAnalyzer()
for label, content in data['Lyrics'].items():
    sentiment.append(analyzer.polarity_scores(content))

In [70]:
neg = []
neu = []
pos = []
compound = []

In [71]:
for sentiments in sentiment:
    neg.append(sentiments['neg'])
    neu.append(sentiments['neu'])
    pos.append(sentiments['pos'])
    compound.append(sentiments['compound'])

In [72]:
data['negative_score'] = neg
data['neutral_score'] = neu
data['positive_score'] = pos
data['compound_score'] = compound

In [73]:
data.isna().sum()

Artist              0
Title               0
Album              42
#_of_views          0
Release_date      166
#_feat_artists      0
Artist_pop          0
Genre               0
Followers           0
Danceability        0
Track_pop           0
Explicit           16
Energy              0
Valence             0
Duration            0
Loudness            0
#_words             0
Lyrics              0
negative_score      0
neutral_score       0
positive_score      0
compound_score      0
dtype: int64

In [74]:
data['Genre'].value_counts()

pop        880
hip hop    700
rock       619
metal      430
country    380
Name: Genre, dtype: int64

# Classification goes below here
We will conduct some experiments on which models to use, and which feature combinations work best with these models.

In [101]:
from sklearn.datasets import load_boston
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 

In [102]:
data.shape

(3009, 22)

In [104]:
X = data.drop(['Genre', 'Artist', 'Title', 'Album', 'Release_date', '#_feat_artists', 'Explicit', 'Lyrics'], axis = 1)
y = data['Genre']

In [105]:
X_train.isna().sum()

#_of_views        0
Artist_pop        0
Followers         0
Danceability      0
Track_pop         0
Energy            0
Valence           0
Duration          0
Loudness          0
#_words           0
negative_score    0
neutral_score     0
positive_score    0
compound_score    0
dtype: int64

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
pipline = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth = 5))
dtree_model = pipeline.fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
cm = confusion_matrix(y_test, dtree_predictions)

In [129]:
for index, record in enumerate(X_test.iterrows()):
    print('Predicted genre is:', dtree_predictions[index], '\nReal genre is:', data.iloc[record[0]]['Genre'], '\n')

Predicted genre is: rock 
Real genre is: rock 

Predicted genre is: hip hop 
Real genre is: hip hop 

Predicted genre is: pop 
Real genre is: country 

Predicted genre is: hip hop 
Real genre is: hip hop 

Predicted genre is: pop 
Real genre is: pop 

Predicted genre is: rock 
Real genre is: pop 

Predicted genre is: metal 
Real genre is: rock 

Predicted genre is: pop 
Real genre is: pop 

Predicted genre is: hip hop 
Real genre is: pop 

Predicted genre is: metal 
Real genre is: rock 

Predicted genre is: hip hop 
Real genre is: hip hop 

Predicted genre is: pop 
Real genre is: rock 

Predicted genre is: pop 
Real genre is: rock 

Predicted genre is: rock 
Real genre is: metal 

Predicted genre is: rock 
Real genre is: rock 

Predicted genre is: pop 
Real genre is: pop 

Predicted genre is: metal 
Real genre is: pop 

Predicted genre is: metal 
Real genre is: metal 

Predicted genre is: metal 
Real genre is: metal 

Predicted genre is: hip hop 
Real genre is: hip hop 

Predicted genr